In [44]:
import numpy as np 
import os
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

In [45]:
path1 = []
path2 = []
path3 = []
path0 = []
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Non Demented'):
    for filename in filenames:
        path0.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Mild Dementia'):
    for filename in filenames:
        path1.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Moderate Dementia'):
    for filename in filenames:
        path3.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Very mild Dementia'):
    for filename in filenames:
        path2.append(os.path.join(dirname, filename))

In [46]:
def create_dict(file_names):
    img_dict = {}
    for name in file_names:
        ind = name[:-8]
        if ind in img_dict:
            img_dict[ind].append(name)
        else:
            dem = name[31:34]
            if dem=='Non':
                img_dict[ind] = [0]
            elif dem == 'Mil':
                img_dict[ind] = [1]
            elif dem == 'Ver':
                img_dict[ind] = [2]
            else:
                img_dict[ind] = [3]
            img_dict[ind].append(name)
    return img_dict
img_dict = create_dict(path0 + path1 + path2 + path3)

In [47]:
def create_df_from_dict(img_dict):
    df = pd.DataFrame({k: v[0] for k,v in img_dict.items()}, index = ['Class']).T
    return df
df_all = create_df_from_dict(img_dict)
df_all.head()

,Class
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0302_MR1_mpr-3,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0114_MR1_mpr-1,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0150_MR1_mpr-3,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0253_MR1_mpr-3,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0349_MR1_mpr-4,0


In [48]:
# Assuming df is your DataFrame and 'class' is your target feature
X = df_all.drop('Class', axis=1)
y = df_all['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [49]:
def create_test_train(img_dict, y_train, y_test):
    X_train_n = []
    y_train_n = []
    X_test_n = []
    y_test_n = []
    for ind in y_test.index:
        paths = img_dict[ind]
        imgs = []
        for j in range(1,len(paths)):
            img = Image.open(paths[j]).resize((128,128))
            img = np.array(img)
            imgs.append(img)
            y_test_n.append(y_test[ind])
        X_test_n.append(imgs)
        
    print('first part done')
    for ind in y_train.index:
        paths = img_dict[ind]
        imgs = []
        for j in range(1,len(paths)):
            img = Image.open(paths[j]).resize((128,128))
            img = np.array(img)
            imgs.append(img)
            y_train_n.append(y_train[ind])
        X_train_n.append(imgs)
    
    
    return X_train_n, X_test_n, y_train_n, y_test_n

In [50]:
X_train, X_test, y_train, y_test = create_test_train(img_dict, y_train, y_test)

first part done


In [5]:
import torch
from torch import nn
import torch.nn.functional as F

# Assuming each image is 128x128x3
input_shape = (128, 128, 3)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.flatten(x)
        return x

# Create 61 CNN models for the 61 input images
cnn_models = [CNNModel() for _ in range(61)]

# Create 61 inputs for the 61 input images
inputs = [torch.randn(input_shape).permute(2,0,1) for _ in range(61)]

# Extract features from each input using the corresponding CNN model
features = [model(inp) for model, inp in zip(cnn_models, inputs)]

# Concatenate all features together
concatenated = torch.cat(features, dim=1)

# Add a dense layer for final prediction
output_layer = nn.Linear(concatenated.size(1), 4)
output = output_layer(concatenated)

# Create the model
model = nn.ModuleList([*cnn_models, output_layer])

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

print(model)

ModuleList(
  (0-60): 61 x CNNModel(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (61): Linear(in_features=242109, out_features=4, bias=True)
)


In [ ]:
for id_images, label in zip(X_train, y_train):
    outputs = model(*id_images)
    loss = nn.CrossEntropyLoss(outputs.unsqueeze(0), label)

# Switch to evaluation mode
model.eval()

# No need to track gradients for testing
with torch.no_grad():
    correct_predictions = 0
    total_predictions = 0

    for id_images, label in zip(X_test, y_test):
        outputs = model(*id_images)
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += label.size(0)
        correct_predictions += (predicted == label).sum().item()

    print('Test Accuracy: %d %%' % (100 * correct_predictions / total_predictions))
